In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from data_acquisition.data_acquisition import  fetch_housing_dataset, fetch_wine_dataset
from models.models import LinearRegression, LogisticRegression
from models.optimizers import GradientDescent, StochasticGradientDescent
from utils.metrics import mse

np.random.seed(0)

## Experiment 1


In [2]:
TEST_SIZE = 0.20
DEFAULT_BATCH_SIZE = 8

housing_df = fetch_housing_dataset(preprocess=True)
wine_df = fetch_wine_dataset()

one_hot_wine_classes = pd.get_dummies(wine_df['class']).to_numpy(dtype=int)
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(wine_df.drop(['class'], axis=1),
                                                                        one_hot_wine_classes,
                                                                        test_size=TEST_SIZE, stratify=wine_df['class'])

X_housing_train, X_housing_test, y_housing_train, y_housing_test = train_test_split(housing_df.drop(['MEDV'], axis=1).to_numpy(),
                                                                                    housing_df.MEDV.to_numpy().reshape(-1,1), test_size=TEST_SIZE)

In [9]:
# Linear regression fit
lin_reg = LinearRegression()

# Analytic fit
lin_reg.fit(X_housing_train, y_housing_train, analytic_fit=True, verbose=False)
y_preds_with_analytic_fit = lin_reg.predict(X_housing_test)
min_ms_error = mse(y_housing_test, lin_reg.predict(X_housing_test))
w_analytic = np.copy(lin_reg.w)

# Mini-batch SGD fit
lin_reg.fit(X_housing_train, y_housing_train, optimizer_class=GradientDescent, batch_size=DEFAULT_BATCH_SIZE, verbose=False)
y_preds_with_grad = lin_reg.predict(X_housing_test)
ms_error = mse(y_housing_test, y_preds_with_grad)


print(f"l2 norm between best weights and SGD weights: {np.linalg.norm(w_analytic - lin_reg.w)}")
print(f"Mean squarred error from analytic fit: {min_ms_error}")
print(f"Mean squared error from SGD with mini-batch size {DEFAULT_BATCH_SIZE}: {ms_error}")

l2 norm between best weights and SGD weights: 0.0014626301535576162
Mean squarred error from analytic fit: 0.008976493178030417
Mean squared error from SGD with mini-batch size 8: 0.008974197239150722


In [11]:
np.random.seed(0)
optimizer_kwargs = {'max_iters': 4e4,
                    'learning_rate': 0.05,
                    'verbose': False}

# Logistic regression fit
log_reg = LogisticRegression()

# Learn model parameters with gradient descent
log_reg.fit(X_wine_train, y_wine_train, optimizer_class=GradientDescent, **optimizer_kwargs)
y_wine_preds_gd = log_reg.predict(X_wine_test)
w_gd = np.copy(log_reg.w)

gd_accuracy = accuracy_score(y_wine_test, y_wine_preds_gd)
gd_recall = recall_score(y_wine_test, y_wine_preds_gd, average='weighted')
gd_precision = precision_score(y_wine_test, y_wine_preds_gd, average='weighted')
gd_f1_score = f1_score(y_wine_test, y_wine_preds_gd, average='weighted')

# Learn model parameters with stochastic gradient descent
log_reg.fit(X_wine_train, y_wine_train, optimizer_class=StochasticGradientDescent, batch_size=DEFAULT_BATCH_SIZE, **optimizer_kwargs)
y_wine_preds_sgd = log_reg.predict(X_wine_test)
w_sgd = np.copy(log_reg.w)

sgd_accuracy = accuracy_score(y_wine_test, y_wine_preds_sgd)
sgd_recall = recall_score(y_wine_test, y_wine_preds_sgd, average='weighted')
sgd_precision = precision_score(y_wine_test, y_wine_preds_sgd, average='weighted')
sgd_f1_score = f1_score(y_wine_test, y_wine_preds_sgd, average='weighted')

print("l2 norm between sgd weights and mini-batch weights: ", np.linalg.norm(w_gd - w_sgd))
print(f"Accuracy:\n \tGD: {gd_accuracy:.5g}, SGD with mini-batch of size {DEFAULT_BATCH_SIZE}: {sgd_accuracy:.5g}")
print(f"Recall:\n \tGD: {gd_recall:.5g}, SGD with mini-batch of size {DEFAULT_BATCH_SIZE}: {sgd_recall:.5g}")
print(f"Precision:\n \tGD: {gd_precision:.5g}, SGD with mini-batch of size {DEFAULT_BATCH_SIZE}: {sgd_precision:.5g}")
print(f"F1 score:\n \tGD: {gd_f1_score:.5g}, SGD with mini-batch of size {DEFAULT_BATCH_SIZE}: {sgd_f1_score:.5g}")

l2 norm between sgd weights and mini-batch weights:  478.5472303883231
Accuracy:
 	GD: 0.88889, SGD with mini-batch of size 8: 0.83333
Recall:
 	GD: 0.88889, SGD with mini-batch of size 8: 0.83333
Precision:
 	GD: 0.91667, SGD with mini-batch of size 8: 0.88333
F1 score:
 	GD: 0.88757, SGD with mini-batch of size 8: 0.82903
